In [1]:
import re, scrapy, os
import pandas as pd
from scrapy.exporters import CsvItemExporter
from scrapy import Spider, Request
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors import LinkExtractor

In [2]:
class item_TeamInfoLinks(scrapy.Item):
    """Scrapy Item for Team Links for 2014-2017"""
    team = scrapy.Field()
    link = scrapy.Field()
    txt = scrapy.Field()
    key = scrapy.Field()
    year = scrapy.Field()

In [3]:
class item_Results(scrapy.Item):
    """Scrapy Item yearly team results"""
    Team = scrapy.Field()
    Date = scrapy.Field()
    Opponent = scrapy.Field()
    Result = scrapy.Field()

In [4]:
class item_TeamStats(scrapy.Item):
    """Scrapy Item yearly team stats"""
    Team = scrapy.Field()
    Stat = scrapy.Field()
    Rank = scrapy.Field()
    Value = scrapy.Field()
    Year = scrapy.Field()

In [5]:
class item_IndividualLeaders(scrapy.Item):
    """Scrapy Item yearly individual leaders"""
    Team = scrapy.Field()
    Stat = scrapy.Field()
    Player = scrapy.Field()
    Value = scrapy.Field()
    Year = scrapy.Field()

In [6]:
class LinksResultsTeamIndividualCSVPipeline(object):
    """Scrapy Pipeline Object for Links, Results, Team, Individual"""
    types = ['teaminfolinks', 'results', 'teamstats', 'individualleaders']
    
    def open_spider(self, spider):
        # CreateFiles 
        self.files = {t: open(os.path.join("Data", t + '.csv'), 'wb') for t in self.types[1:]}
        # Send links to link folder 
        self.files[self.types[0]] = open(os.path.join("Links", self.types[0] + '.csv'), 'wb')
        # Create exporters 
        self.exporters = {t:CsvItemExporter(self.files[t]) for t in self.types}
        [e.start_exporting() for e in self.exporters.values()]
    
    def close_spider(self, spider):
        [e.finish_exporting() for e in self.exporters.values()]
        [f.close() for f in self.files.values()]
 
    def process_item(self, item, spider):
        # Define what item type it is 
        itemType = type(item).__name__.replace("item_", "").lower()
        
        # Export 
        self.exporters[itemType].export_item(item)
        
        return item

In [7]:
class DataSpider(Spider): 
    name = "Data"
    
    # Settings 
    custom_settings = {'DOWNLOAD_DELAY': '.75'}
        
    # Regex patterns 
    patterns = {'people' : ".*/\d+\?sport_code\=MFB",
                'history' : ".*\/teams\/history\/MFB\/\d+",
                'roster' : ".*\/team\/\d+\/roster\/\d+",
                'stats' : ".*\/team\/\d+\/stats\/\d+",
               }
    
    def start_requests(self): 
        """Read in TeamLinks and start crawling"""
        links = pd.read_csv(os.path.join("Links", "TeamLinks.csv"))
        links = links.to_dict(orient='record')
        
        for link in links: 
            yield Request(url=link['Link'], 
                          callback=self.parse,
                          meta= {'REDIRECT_ENABLED': True, 'year':link['Year']}
                         )
    
    def parse(self, response): 
        # Extract Team Name
        team = response.selector.xpath("//body//div//fieldset//legend//a/text()").extract()[0]
        
        # Get all the links on the page
        le = LinkExtractor() 
        links = le.extract_links(response)
                
        # Extract the links pass it to the pipeline for saving
        linksOut = []
        for link in links: 
            for k, pattern in self.patterns.items():
                if re.search(pattern, link.url) != None: 
                    yield item_TeamInfoLinks({'team':team, 
                                              'link':link.url, 
                                              'txt':link.text, 
                                              'key':k, 
                                              'year':response.meta['year']
                                             })
        
        # Find all the tables 
        tables = pd.read_html(response.body)
        
        # Create a results table
        tables[1].rename(columns=tables[1].iloc[1], inplace = True)
        tables[1].drop([0,1], inplace = True)
        tables[1]['Team'] = team
        
        # Convert to table to a list, use dict to create scrapy item, send item to pipeline 
        for record in tables[1].to_dict(orient='record'):
            yield item_Results(record)
                
        # Team stats 
        tables[2].rename(columns=tables[2].iloc[1], inplace = True)
        tables[2].drop([0,1], inplace = True)
        tables[2]['Team'] = team
        tables[2]['Year'] = response.meta['year']
        
        # Convert to table to a list, use dict to create scrapy item, send item to pipeline 
        for record in tables[2][:-1].to_dict(orient='record'):
            yield item_TeamStats(record)
                
        # Individual stats
        tables[3].rename(columns=tables[3].iloc[1], inplace = True)
        tables[3].drop([0,1], inplace = True)
        tables[3]['Team'] = team
        tables[3]['Year'] = response.meta['year']
        
        # Convert to table to a list, use dict to create scrapy item, send item to pipeline 
        for record in tables[3][:-1].to_dict(orient='record'):
            yield item_IndividualLeaders(record)
            

In [ ]:
process = CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
                          'ITEM_PIPELINES': {'__main__.LinksResultsTeamIndividualCSVPipeline':100}
                         })

process.crawl(DataSpider)
process.start() 

2017-12-12 21:22:47 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-12-12 21:22:47 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2017-12-12 21:22:47 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2017-12-12 21:22:47 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'sc

2017-12-12 21:23:10 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66692>
{'Date': '12/30/2016',
 'Opponent': 'South Ala. @ Tucson, Ariz.',
 'Result': 'W 45 - 21',
 'Team': 'Air Force Falcons'}
2017-12-12 21:23:10 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66692>
{'Rank': '36',
 'Stat': 'Total Offense',
 'Team': 'Air Force Falcons',
 'Value': '452.2',
 'Year': 2017}
2017-12-12 21:23:10 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66692>
{'Rank': '3',
 'Stat': 'Rushing Offense',
 'Team': 'Air Force Falcons',
 'Value': '317.4',
 'Year': 2017}
2017-12-12 21:23:10 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66692>
{'Rank': '123',
 'Stat': 'Passing Offense',
 'Team': 'Air Force Falcons',
 'Value': '134.8',
 'Year': 2017}
2017-12-12 21:23:10 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66692>
{'Rank': '8',
 'Stat': 'Team Passing Efficiency',
 

2017-12-12 21:23:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://stats.ncaa.org/teams/66581> (referer: None)
2017-12-12 21:23:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66581>
{'key': 'history',
 'link': 'http://stats.ncaa.org/teams/history/MFB/27',
 'team': 'Appalachian St. Mountaineers',
 'txt': 'Team History',
 'year': 2017}
2017-12-12 21:23:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66581>
{'key': 'people',
 'link': 'http://stats.ncaa.org/people/41727?sport_code=MFB',
 'team': 'Appalachian St. Mountaineers',
 'txt': 'Scott Satterfield',
 'year': 2017}
2017-12-12 21:23:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66581>
{'key': 'roster',
 'link': 'http://stats.ncaa.org/team/27/roster/12424',
 'team': 'Appalachian St. Mountaineers',
 'txt': 'Roster',
 'year': 2017}
2017-12-12 21:23:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66581>
{'key': 

2017-12-12 21:23:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66581>
{'Rank': '69',
 'Stat': 'Kickoff Returns',
 'Team': 'Appalachian St. Mountaineers',
 'Value': '20.65',
 'Year': 2017}
2017-12-12 21:23:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66581>
{'Rank': '42',
 'Stat': 'First Downs Offense',
 'Team': 'Appalachian St. Mountaineers',
 'Value': '291',
 'Year': 2017}
2017-12-12 21:23:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66581>
{'Rank': '19',
 'Stat': 'First Downs Defense',
 'Team': 'Appalachian St. Mountaineers',
 'Value': '232',
 'Year': 2017}
2017-12-12 21:23:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66581>
{'Rank': 'T-62',
 'Stat': 'Fewest Penalties Per Game',
 'Team': 'Appalachian St. Mountaineers',
 'Value': '6.08',
 'Year': 2017}
2017-12-12 21:23:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66581>
{'R

2017-12-12 21:23:12 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66579>
{'Rank': '58',
 'Stat': 'Team Passing Efficiency',
 'Team': 'Akron Zips',
 'Value': '133.53',
 'Year': 2017}
2017-12-12 21:23:12 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66579>
{'Rank': '73',
 'Stat': 'Scoring Offense',
 'Team': 'Akron Zips',
 'Value': '27.4',
 'Year': 2017}
2017-12-12 21:23:12 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66579>
{'Rank': '114',
 'Stat': 'Total Defense',
 'Team': 'Akron Zips',
 'Value': '466.0',
 'Year': 2017}
2017-12-12 21:23:12 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66579>
{'Rank': '107',
 'Stat': 'Rushing Defense',
 'Team': 'Akron Zips',
 'Value': '224.7',
 'Year': 2017}
2017-12-12 21:23:12 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/66579>
{'Rank': '79',
 'Stat': 'Passing Yards Allowed',
 'Team': 'Akron Zips',
 'Value': 

...

2017-12-12 21:30:08 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (302) to <GET http://stats.ncaa.org/teams/62739> from <GET http://stats.ncaa.org/team/418/11520>
2017-12-12 21:30:08 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83595>
{'Date': '09/06/2014',
 'Opponent': '@ Iowa St.',
 'Result': 'W 32 - 28',
 'Team': 'Kansas St. Wildcats'}
2017-12-12 21:30:08 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83595>
{'Date': '09/18/2014',
 'Opponent': 'Auburn',
 'Result': 'L 14 - 20',
 'Team': 'Kansas St. Wildcats'}
2017-12-12 21:30:08 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83595>
{'Date': '09/27/2014',
 'Opponent': 'UTEP',
 'Result': 'W 58 - 28',
 'Team': 'Kansas St. Wildcats'}
2017-12-12 21:30:08 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83595>
{'Date': '10/04/2014',
 'Opponent': 'Texas Tech',
 'Result': 'W 45 - 13',
 'Team': 'Kansas St. Wildcats'}
20

2017-12-12 21:30:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83595>
{'Player': 'Matthew McCrane',
 'Stat': 'FGM',
 'Team': 'Kansas St. Wildcats',
 'Value': '18',
 'Year': 2015}
2017-12-12 21:30:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83595>
{'Player': 'Randall Evans',
 'Stat': 'Int',
 'Team': 'Kansas St. Wildcats',
 'Value': '4',
 'Year': 2015}
2017-12-12 21:30:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83595>
{'Player': 'Joe Hubener',
 'Stat': 'Pass Eff',
 'Team': 'Kansas St. Wildcats',
 'Value': '176.71',
 'Year': 2015}
2017-12-12 21:30:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83595>
{'Player': 'Matthew McCrane',
 'Stat': 'Points',
 'Team': 'Kansas St. Wildcats',
 'Value': '95',
 'Year': 2015}
2017-12-12 21:30:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83595>
{'Player': 'Nick Walsh',
 'Stat': 'Punt Avg',
 'T

2017-12-12 21:30:10 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83597>
{'Rank': '94',
 'Stat': 'Turnover Margin',
 'Team': 'Kent St. Golden Flashes',
 'Value': '-0.36',
 'Year': 2015}
2017-12-12 21:30:10 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83597>
{'Rank': 'T-121',
 'Stat': '3rd Down Conversion Pct',
 'Team': 'Kent St. Golden Flashes',
 'Value': '0.303',
 'Year': 2015}
2017-12-12 21:30:10 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83597>
{'Rank': '18',
 'Stat': '4th Down Conversion Pct',
 'Team': 'Kent St. Golden Flashes',
 'Value': '0.643',
 'Year': 2015}
2017-12-12 21:30:10 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83597>
{'Rank': '111',
 'Stat': '3rd Down Conversion Pct Defense',
 'Team': 'Kent St. Golden Flashes',
 'Value': '0.456',
 'Year': 2015}
2017-12-12 21:30:10 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/83597>
{'

2017-12-12 21:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/62730>
{'Date': '09/28/2013',
 'Opponent': 'Army West Point @ Dallas, Texas',
 'Result': 'L 16 - 35',
 'Team': 'Louisiana Tech Bulldogs'}
2017-12-12 21:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/62730>
{'Date': '10/05/2013',
 'Opponent': '@ UTEP',
 'Result': 'W 38 - 35',
 'Team': 'Louisiana Tech Bulldogs'}
2017-12-12 21:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/62730>
{'Date': '10/19/2013',
 'Opponent': 'North Texas',
 'Result': 'L 13 - 28',
 'Team': 'Louisiana Tech Bulldogs'}
2017-12-12 21:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/62730>
{'Date': '10/26/2013',
 'Opponent': '@ FIU',
 'Result': 'W 23 - 7',
 'Team': 'Louisiana Tech Bulldogs'}
2017-12-12 21:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/62730>
{'Date': '11/09/2013',
 'Opponent': 'S

2017-12-12 21:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/62730>
{'Player': 'Logan McPherson',
 'Stat': 'Punt Avg',
 'Team': 'Louisiana Tech Bulldogs',
 'Value': '450.8',
 'Year': 2014}
2017-12-12 21:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/62730>
{'Player': 'Trent Taylor',
 'Stat': 'Punt Ret Yds',
 'Team': 'Louisiana Tech Bulldogs',
 'Value': '53',
 'Year': 2014}
2017-12-12 21:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/62730>
{'Player': 'DJ Banks',
 'Stat': 'Rec',
 'Team': 'Louisiana Tech Bulldogs',
 'Value': '36',
 'Year': 2014}
2017-12-12 21:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/62730>
{'Player': 'Sterling Griffin',
 'Stat': 'ReceivingYards',
 'Team': 'Louisiana Tech Bulldogs',
 'Value': '357',
 'Year': 2014}
2017-12-12 21:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/teams/62730>
{'Player': 'Kennet